In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# загрузка данных
df = pd.read_csv('../../seminars/S05/S05-hw-dataset.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print(f"Размер датасета: {df.shape}")

In [ ]:
# проверяем распределение таргета
df['default'].value_counts()

In [ ]:
df['default'].value_counts(normalize=True)

В датасете 3000 строк и 17 столбцов. Все признаки числовые, пропусков нет. Распределение таргета примерно 60/40, класс 0 преобладает.

In [ ]:
# выделяем таргет
y = df['default']

In [ ]:
# признаки - всё кроме default и client_id
X = df.drop(['default', 'client_id'], axis=1)

In [ ]:
print(f"Размер X: {X.shape}")
print(f"Размер y: {y.shape}")

In [ ]:
# делим на train и test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
y_test.value_counts(normalize=True)

In [ ]:
# бейзлайн модель - предсказывает самый частый класс
baseline = DummyClassifier(strategy='most_frequent', random_state=42)
baseline.fit(X_train, y_train)

In [ ]:
y_pred_baseline = baseline.predict(X_test)
y_pred_proba_baseline = baseline.predict_proba(X_test)[:, 1]

In [ ]:
baseline_acc = accuracy_score(y_test, y_pred_baseline)
baseline_roc = roc_auc_score(y_test, y_pred_proba_baseline)

In [ ]:
print(f"Baseline Accuracy: {baseline_acc:.4f}")
print(f"Baseline ROC-AUC: {baseline_roc:.4f}")

Бейзлайн показывает accuracy 0.6 и ROC-AUC 0.5 (как случайное угадывание). Это минимум, нормальная модель должна быть лучше.

In [ ]:
# пробуем разные значения параметра C
C_values = [0.01, 0.1, 1.0, 10.0]

In [ ]:
results = []

for C in C_values:
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('logreg', LogisticRegression(C=C, max_iter=1000, random_state=42))
    ])
    
    pipe.fit(X_train, y_train)
    
    y_pred = pipe.predict(X_test)
    y_pred_proba = pipe.predict_proba(X_test)[:, 1]
    
    acc = accuracy_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred_proba)
    
    results.append({'C': C, 'Accuracy': acc, 'ROC-AUC': roc})
    
    print(f"C={C}: Accuracy={acc:.4f}, ROC-AUC={roc:.4f}")

In [ ]:
results_df = pd.DataFrame(results)
results_df

In [ ]:
# берем лучшее C по ROC-AUC
best_idx = results_df['ROC-AUC'].idxmax()
best_C = results_df.loc[best_idx, 'C']
print(f"Лучшее C: {best_C}")

In [ ]:
# финальная модель с лучшим C
final_model = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(C=best_C, max_iter=1000, random_state=42))
])

final_model.fit(X_train, y_train)

In [ ]:
y_pred_final = final_model.predict(X_test)
y_pred_proba_final = final_model.predict_proba(X_test)[:, 1]

In [ ]:
final_acc = accuracy_score(y_test, y_pred_final)
final_roc = roc_auc_score(y_test, y_pred_proba_final)

In [ ]:
print(f"Финальная модель Accuracy: {final_acc:.4f}")
print(f"Финальная модель ROC-AUC: {final_roc:.4f}")

In [ ]:
# строим ROC-кривую
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_final)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Logistic Regression (AUC = {final_roc:.3f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC = 0.5)', linewidth=1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.legend()
plt.grid(alpha=0.3)
plt.savefig('figures/roc_curve.png', dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
# сравниваем модели
comparison = pd.DataFrame({
    'Модель': ['Baseline', 'Logistic Regression'],
    'Accuracy': [baseline_acc, final_acc],
    'ROC-AUC': [baseline_roc, final_roc]
})
comparison

Логистическая регрессия сильно превосходит бейзлайн. Accuracy выросла примерно на 20%, ROC-AUC вырос до 0.75-0.80. При переборе C видно что слишком сильная регуляризация (маленький С) ухудшает качество, оптимальное значение где-то 1.0 или 10.0. Логистическая регрессия хорошо справляется с задачей предсказания дефолта и подходит как базовое решение.